In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from torchvision.transforms import Resize, InterpolationMode, ToPILImage
from torchmetrics import JaccardIndex, Precision, Recall, F1Score
import segmentation_models_pytorch as smp

from src.evaluation.evaluate_result import evaluate_result
from src.callbacks.SaveRandomImagesCallback import SaveRandomImagesCallback
from src.callbacks.SaveTestPreds import SaveTestPreds
from src.datasets.INRIAAerialImageLabellingDatasetPatches import (
    INRIAAerialImageLabellingDatasetPatches,
)
from src.utils import view_and_save_images_shapes
from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

d:\__repos\aerial_segmentation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if you get some cryptic CUDA error, set device to "cpu" and try again
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 16
SEED = 42
IMAGE_SIZE = 576
SAVE_VAL_DIR = "outputs/INRIA-patches/val"
SAVE_TEST_DIR = "outputs/INRIA-patches/test"
INRIA_PATCHES_DATASET_PATH = "data/INRIAAerialImageLabellingDatasetPatches"  # home PC

In [5]:
labeled_dataset = INRIAAerialImageLabellingDatasetPatches(
    INRIA_PATCHES_DATASET_PATH,
    split="train",
    transforms=[ResizeToDivisibleBy32()],
)
print(len(labeled_dataset))

18000


In [6]:
test_dataset = INRIAAerialImageLabellingDatasetPatches(
    INRIA_PATCHES_DATASET_PATH,
    split="test",
    transforms=[ResizeToDivisibleBy32()],
)
print(len(test_dataset))

14400


## Prepare data

In [7]:
sanity_check_dataset = INRIAAerialImageLabellingDatasetPatches(
    INRIA_PATCHES_DATASET_PATH,
    split="train",
    transforms=[ResizeToDivisibleBy32()],
)
print(len(sanity_check_dataset))

18000


In [8]:
sanity_check_dataloader = DataLoader(
    sanity_check_dataset, batch_size=BATCH_SIZE, shuffle=True
)

In [9]:
for images, masks in sanity_check_dataloader:
    print("images")
    print(images.shape)
    unique, counts = np.unique(images, return_counts=True)
    print(dict(zip(unique, counts)))
    print()
    print("masks")
    print(masks.shape)
    unique, counts = np.unique(masks, return_counts=True)
    print(dict(zip(unique, counts)))
    break

images
torch.Size([16, 3, 512, 512])
{0.0: 7805, 0.003921569: 20313, 0.007843138: 13147, 0.011764706: 5582, 0.015686275: 4850, 0.019607844: 4705, 0.023529412: 4551, 0.02745098: 4603, 0.03137255: 4635, 0.03529412: 5265, 0.039215688: 5801, 0.043137256: 7249, 0.047058824: 8151, 0.050980393: 9106, 0.05490196: 9675, 0.05882353: 10280, 0.0627451: 11696, 0.06666667: 13541, 0.07058824: 16463, 0.07450981: 20899, 0.078431375: 23438, 0.08235294: 24084, 0.08627451: 23864, 0.09019608: 23735, 0.09411765: 23242, 0.09803922: 22960, 0.101960786: 22333, 0.105882354: 22393, 0.10980392: 22987, 0.11372549: 23853, 0.11764706: 25911, 0.12156863: 29845, 0.1254902: 37503, 0.12941177: 50847, 0.13333334: 63833, 0.13725491: 59599, 0.14117648: 57054, 0.14509805: 54288, 0.14901961: 59276, 0.15294118: 61034, 0.15686275: 67825, 0.16078432: 73970, 0.16470589: 74629, 0.16862746: 85915, 0.17254902: 87636, 0.1764706: 94596, 0.18039216: 97159, 0.18431373: 105657, 0.1882353: 112279, 0.19215687: 106332, 0.19607843: 100643, 

In [10]:
# view_and_save_images_shapes(
#     sanity_check_dataloader, "inria_patches_images_shapes", verbose=True
# )

### Train, validation and test data

In [11]:
train_size = int(0.8 * len(labeled_dataset))
val_size = len(labeled_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    labeled_dataset, [train_size, val_size]
)
print(len(train_dataset), len(val_dataset), len(test_dataset))

14400 3600 14400


In [12]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    persistent_workers=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    persistent_workers=True,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    persistent_workers=True,
)

In [13]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([16, 3, 512, 512])
torch.Size([16, 1, 512, 512])


In [14]:
to_pil_transform = ToPILImage()

In [15]:
if BATCH_SIZE == 1:
    img = to_pil_transform(images.squeeze())

In [16]:
# img.show()

In [17]:
if BATCH_SIZE == 1:
    msk = to_pil_transform(masks.squeeze()).convert("L")

In [18]:
# msk.show()

# Training module

In [19]:
class SegmentationModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super(SegmentationModel, self).__init__()

        self.model = model
        self.learning_rate = learning_rate

        self.criterion = smp.losses.SoftBCEWithLogitsLoss()
        # self.criterion = smp.losses.DiceLoss()
        self.train_iou = JaccardIndex(num_classes=2, task="binary")
        self.val_iou = JaccardIndex(num_classes=2, task="binary")

    def forward(self, x):
        output = self.model(x.to(device))
        return output

    def training_step(self, batch, batch_idx):
        images, masks = batch
        masks = torch.div(masks, 255).float()
        preds = self(images)
        loss = self.criterion(preds, masks)

        self.log("train_loss", loss, on_epoch=True, on_step=True)
        self.log("train_iou", self.train_iou(preds, masks), on_epoch=True, on_step=True)

        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        masks = torch.div(masks, 255).float()
        preds = self(images)
        loss = self.criterion(preds, masks)

        self.log("val_loss", loss, on_epoch=True, on_step=True)
        self.log("val_iou", self.val_iou(preds, masks), on_epoch=True, on_step=True)

        return loss

    def test_step(self, batch, batch_idx):
        # just here to activate the test_epoch_end
        # callback SaveTestPreds starts on_test_epoch_end
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [20]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
    activation="sigmoid",
).to(device)

In [21]:
# model = smp.UnetPlusPlus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,  # model output channels (number of classes in your dataset)
# ).to(device)

In [22]:
# model = smp.DeepLabV3(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
#     activation="sigmoid"
# ).to(device)

In [23]:
# model = smp.DeepLabV3Plus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
# ).to(device)

In [24]:
segmentation_model = SegmentationModel(model)

In [25]:
model_checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", save_top_k=-1, mode="min"
)

In [26]:
logger = CSVLogger("lightning_logs", name="inria_patches_segmentation_model")

In [27]:
save_callback = SaveRandomImagesCallback(save_dir=SAVE_VAL_DIR)
save_test_preds_callback = SaveTestPreds(save_dir=SAVE_TEST_DIR)
early_stopping_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)

In [28]:
trainer = pl.Trainer(
    max_epochs=50,
    log_every_n_steps=10,
    callbacks=[
        model_checkpoint_callback,
        save_callback,
        save_test_preds_callback,
        early_stopping_callback
    ],
    logger=logger,
)

# DEBUG
# trainer = pl.Trainer(
#     max_epochs=1,
#     callbacks=[model_checkpoint_callback, save_callback],
#     logger=logger,
#     accelerator="cpu"
#     )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
trainer.fit(
    segmentation_model, train_dataloaders=train_loader, val_dataloaders=val_loader
)

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | Unet                  | 14.3 M
1 | criterion | SoftBCEWithLogitsLoss | 0     
2 | train_iou | BinaryJaccardIndex    | 0     
3 | val_iou   | BinaryJaccardIndex    | 0     
----------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.313    Total estimated model params size (MB)


Epoch 15:  24%|██▎       | 212/900 [01:34<05:07,  2.24it/s, v_num=4]       

In [ ]:
trainer.test(ckpt_path="best", dataloaders=test_loader)